In [46]:
%matplotlib inline

import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve,average_precision_score
from sklearn.metrics import mean_squared_error,r2_score
from scipy.sparse import csr_matrix, csc_matrix
import random as random
from sklearn.decomposition import NMF,TruncatedSVD
#Fit the code to the screen
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

### Reading the ratings data

In [18]:
#Reading the ratings data
my_path = !pwd
ratings_filename = "/ml-1m/ratings.dat"
dfratings = pd.read_csv(my_path[0]+ratings_filename, index_col=None, sep='::', header=None)
dfratings.columns = ["userId", "movieId", "rating", "timestamp"]

/tank/home/adif/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.


### Creating ratings sparse matrix

In [26]:
#Split to train and test sets
test_idx = np.array(random.sample(range(0,len(dfratings)), int(len(dfratings)/10)))
train_idx = set(np.arange(len(dfratings))) - set(test_idx)
train_idx = list(train_idx)
dfratings_test = dfratings.iloc[test_idx]
dfratings_train = dfratings.iloc[train_idx]
i=0
for i1,rows in dfratings_test.iterrows():
    print rows
    i+=1
    if i>5:
        break

userId            1083
movieId           2085
rating               5
timestamp    974934691
Name: 170137, dtype: int64
userId             169
movieId              3
rating               5
timestamp    977202633
Name: 23422, dtype: int64
userId            6001
movieId           3556
rating               4
timestamp    960441181
Name: 993630, dtype: int64
userId            2164
movieId           1272
rating               5
timestamp    974615992
Name: 369729, dtype: int64
userId            5716
movieId           2019
rating               5
timestamp    960325035
Name: 946445, dtype: int64
userId            4433
movieId           1748
rating               5
timestamp    965099448
Name: 741942, dtype: int64


In [99]:
users = (dfratings_train["userId"]).unique()
movies = (dfratings_train["movieId"]).unique()
users_num = max(users)
movies_num = max(movies)
users_num += 1
movies_num += 1

In [51]:
predictions_dict = dict()
train_csr = csr_matrix((dfratings_train['rating'],(dfratings_train['userId'],dfratings_train['movieId'])), shape=(users_num, movies_num), dtype=float)
nmf_components_list = [4,8,16,32]
for nmf_components in nmf_components_list:
    nmf = NMF(n_components=nmf_components) 
    us = nmf.fit_transform(train_csr)
    vt = nmf.components_
    pred = [np.dot(us[row['userId'],:],vt[:,row['movieId']])  for index,row in dfratings_test.iterrows()]
    for idx in range(len(pred)):
        if pred[idx]<1:
            pred[idx]=1
        if pred[idx]>5:    
            pred[idx]=5
    predictions_dict["NMF{0}".format(nmf_components)] = pred

svd_components_list = [4,8,16,32]
for svd_components in svd_components_list:
    svd = TruncatedSVD(n_components=svd_components, random_state=42) 
    us = svd.fit_transform(train_csr)
    vt = svd.components_
    pred = [np.dot(us[row['userId'],:],vt[:,row['movieId']])  for index,row in dfratings_test.iterrows()]
    for idx in range(len(pred)):
        if pred[idx]<1:
            pred[idx]=1
        if pred[idx]>5:    
            pred[idx]=5
    predictions_dict["SVD{0}".format(svd_components)] = pred


In [52]:
label = dfratings_test['rating'] # the real ratings
for predictor in predictions_dict.keys():
    print predictor, mean_squared_error(predictions_dict[predictor],label)


SVD32 5.45202901691
SVD4 6.26736789526
SVD16 5.61379991596
SVD8 5.89273567974


In [ ]:
#Maybe we want to use this: train_csr.eliminate_zeros()